In [ ]:
import string
import pandas as pd
from imblearn.over_sampling import SMOTE, RandomOverSampler
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.utils.data
import numpy as np
from keras.preprocessing.text import one_hot
import warnings
warnings.filterwarnings("ignore")
'''
About the task:

You are provided with a codeflow- which consists of functions to be implemented(MANDATORY).

You need to implement each of the functions mentioned below, you may add your own function parameters if needed(not to main).
Execute your code using the provided auto.py script(NO EDITS PERMITTED) as your code will be evaluated using an auto-grader.

Introduce hidden layer(s),
	Use sigmoid as well as RELU for the hidden neurons and study the difference,
	Develop a good GUI for input mechanism,
	Use word2vec, GloVe, FastText for word embedding and compare results,
	Solve the data unbalance problem (class V has 65% of the data),
	Achieve at least 85% accuracy (currently for many groups it is in the range of 40s).
'''

num_words = 29
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

def encode_data(text):
    # This function will be used to encode the reviews using a dictionary(created using corpus vocabulary) 
    
    # Example of encoding :"The food was fabulous but pricey" has a vocabulary of 4 words, each one has to be mapped to an integer like: 
    # {'The':1,'food':2,'was':3 'fabulous':4 'but':5 'pricey':6} this vocabulary has to be created for the entire corpus and then be used to 
    # encode the words into integers 

    corpus = []
    for lines in text['reviews']:
      for word in lines:
        corpus.append(word)
    unique_words = set(corpus)
    #vocab_length = len(corpus_unique_words)
    vocab_length = len(unique_words)
    # print(vocab_length)
    for i in range(0,len(text['reviews'])):
      line = (" ").join(text['reviews'].iloc[i])
      text['reviews'].iloc[i] = one_hot(line,vocab_length)

    return text


def convert_to_lower(text):
    # return the reviews after convering then to lowercase
    return text.apply(lambda row :row.lower())


def remove_punctuation(text):
    # return the reviews after removing 
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text


def remove_stopwords(text):
    # return the reviews after removing the stopwords
    # print('before stopword removal',text)
    stop = stopwords.words('english')
    return text.apply(lambda x: (" ").join([item for item in x.split() if item not in stop]))
    

def perform_tokenization(text):
    # return the reviews after performing tokenization
    return text.apply(lambda row : word_tokenize(row))


def perform_padding(data):
    # return the reviews after padding the reviews to maximum length
    pd.options.mode.chained_assignment = None  # default='warn'
    #review_max_length = data['reviews'].str.len().max()
    max_length = 29
    for i in range(0,len(data['reviews'])):
        data['reviews'].iloc[i] += ['0']*(max_length - len(data['reviews'].iloc[i]))
        data['reviews'].iloc[i] = np.array(data['reviews'].iloc[i]).astype('int64')
    return data

def preprocess_data(data):
    # make all the following function calls on your data
    # EXAMPLE:->
    

    review = data["reviews"]
    
    review = convert_to_lower(review)
    review = review.apply(remove_punctuation)
    data["reviews"] = review
    review = remove_stopwords(review)
    # data = remove_stopwords(data)
    review = perform_tokenization(review)
    data["reviews"] = review
    data = encode_data(data)
    review = data["reviews"]
    data = perform_padding(data)
    
    return data

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import torch.nn as nn
import nltk
nltk.download('punkt')
from sklearn.metrics import classification_report
from math import log2

def softmax_activation(x):
    # write your own implementation from scratch and return softmax values(using predefined softmax is prohibited)
    # print((torch.exp(x) - torch.max(torch.exp(x)))/ (torch.sum(torch.exp(x),axis=0) - torch.max(torch.exp(x))))
    x=x-torch.max(x)
    return torch.exp(x)/(torch.sum(torch.exp(x),axis=0))

class NeuralNet(nn.Module):

    def __init__(self,train_data_loader,val_data_loader):
        super(NeuralNet, self).__init__()
        
        #1st layer
        self.layer1 = nn.Linear(num_words,16)
        self.non_linear_layer1 = nn.Sigmoid()
        #2nd layer
        # self.layer2 = nn.Linear(16,16)
        # self.non_linear_layer2 = nn.Sigmoid()
        #final layer
        self.layer_final = nn.Linear(16,5) 
        #initializing loaded input datasets
        self.train_data_loader = train_data_loader
        self.val_data_loader = val_data_loader


    def build_nn(self,dt):
        #add the input and output layer here; you can use either tensorflow or pytorch
        x1 = self.layer1(dt)
        x1 = self.non_linear_layer1(x1)
        # x1 = self.layer2(x1)
        # x1 = self.non_linear_layer2(x1)
        x1 = self.layer_final(x1)
        x1 = softmax_activation(x1)
        return x1

    # calculate cross entropy
    # def cross_entropy(p, q):
	  #     return -sum([p[i]*log2(q[i]) for i in range(len(p))])
   

    def train_nn(self,batch_size,epochs,optimizer,model):
        # write the training loop here; you can use either tensorflow or pytorch
        loss_cr = nn.CrossEntropyLoss()
        train_ls = []
        val_ls  = []
        train_acc = []
        val_acc = []
        
        for epoch in range(epochs):
            #for param in model.parameters():
            #  print(param.data)
            # For training
            tot_loss = 0
            correct = 0
            total = 0
            for (batch,label) in self.train_data_loader:
                data = batch
                labels = label
                optimizer.zero_grad()
                output = model.build_nn(data)
                loss = loss_cr(output,labels)
                #summing the loss
                tot_loss = loss.item()
                #For backpropagation
                loss.backward()
                optimizer.step()
                total += labels.size(0)
                _,pred = torch.max(output.data,1)
                correct += (pred == labels).sum().item()
            train_acc1 = 100 * correct / total
            train_acc.append(train_acc1)
            
            # print validation accuracy
            val_loss=0
            # We shall use the validation set to tune the hyperparameters that we use
            model.eval()
            correct = 0
            total = 0
            #Now, similar to above, we take batch  from validation data, one batch containing 16 examples in one iteration
            for (batch,label) in self.val_data_loader:
                data = batch
                labels = label
                output = model.build_nn(data)
                loss = loss_cr(output,labels)
                val_loss = loss.item()
                total += labels.size(0)
                _,pred = torch.max(output.data,1)
                correct += (pred == labels).sum().item()
            valid_acc = 100 * correct / total
            val_acc.append(valid_acc)


            train_ls.append(tot_loss)
            val_ls.append(val_loss)
          
            
            print(f'Epoch {epoch} --> Train loss: {tot_loss} Train accuracy: {train_acc[-1]} Val loss: {val_loss} Val accuracy: {val_acc[-1]}')
            
        return 0
        
    def predict(self, data,model1):
        output = model1.build_nn(data)
        pred,index = torch.max(output,1)
        
        return index + 1,self.layer1
        # return a list containing all the ratings predicted by the trained model


# DO NOT MODIFY MAIN FUNCTION'S PARAMETERS

def main(train_file, test_file):
    global train_dataset
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)

    batch_size = 150
    epochs = 50
    
    

    train_dataset=preprocess_data(train_data) 
    train_dataset = train_dataset.loc[:,~train_dataset.columns.str.match("Unnamed")]
    test_dataset=preprocess_data(test_data)
    test_dataset = test_dataset.loc[:,~test_dataset.columns.str.match("Unnamed")]
    print(train_dataset)
    X = torch.tensor(train_dataset['reviews'], dtype=torch.float)
    X = (X - torch.mean(X))/torch.std(X)
    y = torch.tensor(train_dataset['ratings'], dtype=torch.float) -1
    
#     y = y - torch.tensor(1).expand_as(y)
    y = y.to(torch.int64)
    X_train,X_val,y_train, y_val = train_test_split(X,y,test_size=0.3,shuffle=True)
    sm = RandomOverSampler()
    X1,y1 = sm.fit_resample(X_train,y_train)
    X1 = torch.tensor(X1,dtype=torch.float)
    y1 = torch.tensor(y1,dtype=torch.int64)
    train_dataset = torch.utils.data.TensorDataset(X1,y1)
    # train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
    validation_dataset = torch.utils.data.TensorDataset(X_val,y_val)
    print(train_dataset)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
    test_dataset = torch.tensor(test_dataset['reviews'], dtype=torch.float)
    test_dataset = (test_dataset - torch.mean(test_dataset))/torch.std(test_dataset)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)
    
    model=NeuralNet(train_loader,val_loader)
    # model.build_nn()
    lr = 0.01
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    model.train_nn(batch_size,epochs,optimizer,model)
    output,lay = model.predict(test_dataset,model)
    output = output.cpu().detach().numpy()
    print('output:',output)
    return output,model,lay

val = []
func = ''
lay = ''
if __name__ == "__main__":
    val,func,lay=main("train.csv","test.csv")
    original_data = pd.read_csv("gold_test.csv")['ratings']
    print(val)
    print(classification_report(original_data, val))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
                                                 reviews  ratings
0      [1786, 3272, 18769, 8406, 6882, 0, 0, 0, 0, 0,...        4
1      [16579, 1296, 16195, 12840, 11961, 16867, 4968...        5
2      [17631, 5166, 19152, 15849, 536, 13766, 7482, ...        1
3      [3166, 8640, 19152, 992, 10763, 3881, 8752, 60...        5
4      [13634, 278, 17524, 536, 10012, 4711, 9053, 0,...        5
...                                                  ...      ...
49995  [18435, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...        1
49996  [9246, 1456, 4210, 848, 18875, 5213, 5213, 120...        1
49997  [12805, 3788, 17233, 15345, 15878, 12403, 1407...        1
49998  [19152, 9039, 14078, 2041, 5213, 6321, 882, 69...        1
49999  [9648, 1405, 18470, 15216, 10866, 825, 16550, ...        5

[50000 rows x 2 columns]
Epoch 0 --> Train loss: 1.6094071865081787 Train accuracy: 21.513727856

In [ ]:
import os
os.chdir('..')

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()

In [ ]:
train_data = pd.read_csv('train.csv')
train_dataset = preprocess_data(train_data) 
train_dataset = train_dataset.loc[:,~train_dataset.columns.str.match("Unnamed")]

In [ ]:
X = torch.tensor(train_dataset['reviews'], dtype=torch.float)
X = (X - torch.mean(X))/torch.std(X)
y = torch.tensor(train_dataset['ratings'], dtype=torch.float)-1

In [ ]:
X_train,X_val,y_train, y_val = train_test_split(X,y,test_size=0.3,shuffle=True)

In [ ]:
X1,y1 = sm.fit_resample(X_train,y_train)

In [ ]:
X1 = torch.tensor(X1,dtype=torch.float)
y1 = torch.tensor(y1,dtype=torch.int)
train_dataset = torch.utils.data.TensorDataset(X1,y1)

In [ ]:
from sklearn.metrics import f1_score,accuracy_score
original_data = pd.read_csv("gold_test.csv")['ratings']
f1_score(original_data, val,average='weighted')
# accuracy_score(original_data, val)

0.2255483892120651

In [ ]:
pd.read_csv('train.csv')

,Unnamed: 0,reviews,ratings
0,0,"This book was very informative, covering all a...",4
1,1,I am already a baseball fan and knew a bit abo...,5
2,2,I didn't like this product it smudged all unde...,1
3,3,I simply love the product. I appreciate print ...,5
4,4,It goes on very easily and makes my eyes look ...,5
...,...,...,...
49995,49995,it does not work((((((((((((,1
49996,49996,"Really worthless, loud motor with absolutely n...",1
49997,49997,Don't waste your money on this. It does nothin...,1
49998,49998,"Product does not remove ear wax. No suction, j...",1
